# Расчет метрик, которые сделала модель Gemini 3 Pro в 60 ответах полученных через чат

In [1]:
import polars as pl
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
f = "mbti_gemini_results.csv"

# 2. Основная функция расчета метрик с использованием Polars
all_metrics_data = []

def calculate_metrics_polars(csv_file, task_name, dataset_name):
    # Читаем CSV с помощью Polars
    df = pl.read_csv(csv_file)
    
    # 1. Фильтрация данных (валидация)
    # Оставляем строки, где предсказание не 'XXXX' и длина строки равна 4
    # Также убедимся, что ground truth валиден
    valid_df = df.filter(
        (pl.col("type_gemini_pred") != "XXXX") & 
        (pl.col("type_gemini_pred").str.len_chars() == 4) &
        (pl.col("type").str.len_chars() == 4)
    )
    
    n = valid_df.height
    
    if n == 0:
        print(f"[{dataset_name}] No valid predictions to calculate metrics.")
        return

    # Извлекаем списки для sklearn (так как F1 проще считать через sklearn)
    y_true_clean = valid_df["type"].to_list()
    y_pred_clean = valid_df["type_gemini_pred"].to_list()

    print(f"\n--- Metrics for {task_name} (N={n}) ---")
    
    # 2. Основные метрики (Exact Match и F1)
    acc = accuracy_score(y_true_clean, y_pred_clean)
    _, _, f1_macro, _ = precision_recall_fscore_support(
        y_true_clean, y_pred_clean, average='macro', zero_division=0
    )

    print(f"Exact Match Accuracy: {acc:.2%}")
    print(f"Macro F1-Score:       {f1_macro:.2%}")

    # 3. Расчет метрик по осям MBTI (используем Polars Expressions для скорости)
    axes = ["(I)E", "(N)S", "(T)F", "(J)P"]
    axis_scores = []
    
    # Создаем колонки совпадений для каждой буквы (Axis 0, 1, 2, 3)
    # Например: сравниваем первую букву type с первой буквой type_pred
    expressions = []
    for i in range(4):
        # Выражение: True если i-й символ совпадает
        expr = (
            pl.col("type").str.slice(i, 1) == pl.col("type_gemini_pred").str.slice(i, 1)
        ).alias(f"match_{i}")
        expressions.append(expr)
    
    # Добавляем колонки с булевыми значениями совпадений
    df_matches = valid_df.with_columns(expressions)

    # Считаем Total Letters Correct (сумма всех совпадений по всем осям для всех строк)
    # Сначала суммируем совпадения внутри строки (0-4), потом берем сумму по всему датафрейму
    df_matches = df_matches.with_columns(
        sum([pl.col(f"match_{i}").cast(pl.Int8) for i in range(4)]).alias("row_correct_count")
    )
    
    total_letters_correct = df_matches["row_correct_count"].sum()
    
    # Вывод точности по каждой оси
    for i in range(4):
        # Количество True в колонке match_i
        correct_count = df_matches[f"match_{i}"].sum()
        axis_acc = correct_count / n
        axis_scores.append(axis_acc)
        print(f"Axis {axes[i]} Accuracy:      {axis_acc:.2%}")

    # 4. Среднее количество угаданных букв
    avg_letters = total_letters_correct / n
    print(f"Avg Letters Correct:  {avg_letters:.2f} / 4.00")
    
    all_metrics_data.append({
        "Dataset": dataset_name,
        "Exact_Acc": acc,
        "Macro_F1": f1_macro,
        "Avg_Letters": avg_letters
    })

    return all_metrics_data

calculate_metrics_polars(f, "MBTI Prediction Task", "test_csv_data")

print("\nReturned Data Structure:")
print(all_metrics_data)


--- Metrics for MBTI Prediction Task (N=50) ---
Exact Match Accuracy: 74.00%
Macro F1-Score:       56.95%
Axis (I)E Accuracy:      98.00%
Axis (N)S Accuracy:      88.00%
Axis (T)F Accuracy:      88.00%
Axis (J)P Accuracy:      86.00%
Avg Letters Correct:  3.60 / 4.00

Returned Data Structure:
[{'Dataset': 'test_csv_data', 'Exact_Acc': 0.74, 'Macro_F1': 0.5694928880643165, 'Avg_Letters': 3.6}]
